![Image of Yaktocat](https://img-blog.csdnimg.cn/2019072411484875.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3UwMTQ0NTM4OTg=,size_16,color_FFFFFF,t_70)

<img src="https://pic3.zhimg.com/v2-763773acb7949d52c2a1fdd410dd8cf2_r.jpg" width="1500">

In [1]:
import torch
from torch import nn as nn

In [2]:
"""
由于所有类型的ResNet的卷积核都只有1*1， 3*3两种，
所以先定义卷积核
"""
def conv3x3(in_channel, out_channel, stride=1, groups=1, dilation=1):
    return nn.conv2d(in_channel=in_channel,    # 输入通道数
                     out_channel=out_channel,  # 输出通道数
                     kernel_size=3,            # 卷积核大小
                     stride=stride,            # 步长 
                     padding=dilation,         # 填充
                     groups=groups,            # 从输入通道到输出通道的阻塞连接数，默认值：1
                     bisa=FFalse,              # 偏置，为输出添加可学习的偏差。默认值：真
                     dilation=dilation,        # 内核元素之间的间距。默认值：1
                    )

def conv1x1(in_channel, out_channel, stride=1):
    return nn.conv2d(in_channel=in_channel,
                     out_channel=out_channel,
                     kernel_size=3,
                     stride=stride,
                    )

In [3]:
"""
定义BasicBlock
"""
class BasicBlock(nn.Module):
    expansion = 1 # expansion是BasicBlock和Bottleneck的核心区别之一
    
    def __init__(self, in_channel, out_channel, stride=1, downsample=None, groups=1, base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d                        # 未指定标准层时使用pytorch api提供的BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        
        self.conv1 = conv3x3(in_channel, out_channel, stride)   # BasicBlock所有卷积核大小都是3*3，且两层卷积层通道数都是一样的
        self.bn1 = norm_layer(out_channel)                      
        self.relu = nn.ReLU(inplace=True)                       # inplace为True表示进行原地操作，一般默认为False，表示新建一个变量存储操作
        
        self.conv2 = conv3x3(out_channel, out_channel)          # 和第一层卷积层通道数保持一致
        self.bn2 = norm_layer(out_channel)                      
        
        self.downsample = downsample                            
        self.stride = stride
    
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)                                    
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)                                   
        out = self.bn2(out)
        if self.downsample is not None:                         # 下采样，采用1*1的卷积层变成想要的通道数。这是为了保证原始输入与卷积后的输出层叠加时维度相同
            identity = self.downsample(x)
        out += identity                                         
        out = self.relu(out)
        
        return out

In [4]:
"""
定义Bottleneck
"""
class Bottleneck(nn.Module):
    expansion = 4   # Battleneck 输出通道数是输入通道数4倍
    
    def __init__(self, in_channel, out_channel, stride=1, downsample=None, groups=1, base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(out_channel * (base_width / 64.)) * groups
        
        self.conv1 = conv1x1(in_channel, width)
        self.bn1 = norm_layer(width)
        
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        
        self.conv3 = conv1x1(width, out_channel * self.expansion)
        self.bn3 = norm_layer(out_channel * self.expansioan)
        
        self.relu = nn.ReLU(inplace=True)
        
        self.downsample = downsample
        self.stride = stride
       
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        out = self.conv3(out)
        out = slef.bn3(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        
        return out

In [5]:
"""
定义ResNet类
"""
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self.norm_layer = norm_layer
        self.out_channel = 64     # 定义类中的输入通道数
        self.dilation = 1        # 定义类中的内核元素间距
        
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        
        self.groups = groups
        self.base_width = width_per_group
        
        self.conv1 = nn.Conv2d(3, self.out_channel, 7, stride=2, padding=3, bias=False)
        self.bn = norm_layer(self.out_channel)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer3 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        
        self.globepool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc = nn.Linear(512 * block.expansion, num_classes)
 
        for m in self.modules():                       # 初始化定义方法
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)
 
    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )
 
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))
 
        return nn.Sequential(*layers)
 
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
 
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
 
        x = self.avgpool(x)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
 
        return x


![Image of Yaktocat](https://img-blog.csdnimg.cn/2019072411415480.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3UwMTQ0NTM4OTg=,size_16,color_FFFFFF,t_70)